# Reinforcement Learning Project

#### Adrien DANEL & Slimane THABET

This notebook shows our work related to this paper : [**Can Deep Reinforcement Learning Solve Erdos-Selfridge-Spencer Games?**](https://arxiv.org/abs/1711.02301)

## Paper description

The paper points out that developing learning algorithms that are robust across tasks and policy representations remains a
challenge. While standard benchmarks like MuJoCo and Atari provide rich settings for experimentation, but the specifics of the underlying environments differ from each other in multiple ways, and thus determining the principles underlying any particular form of sub-optimal behavior is difficult. To evaluate the success of the alorithms we try on those environments, we usually uses **high scores** to determine those which are more performant, but it would be more interesting to look at **optimal behavior**, which is unfortunately not fully characterized and generally complex on this type of environment.

The article proposes an new environment to better evaluate generalization, it is based on the work of Erdos and Selfridge, and Spencer. Erdos-Selfridge-Spencer (ESS) games are turn-based games where two players play adversarly. The cool thing is that **optimal behavior** (or strategy) can be defined by **potential functions** derived from conditional expectations over random future play.

They chose to focus on a very well-known game type in the Reinforcement Learning, the **attacker-defender game**.

The game is composed, at the beginning, of $N$ identical pieces ditributed on a board of $K$ levels. We are only interested of the level of each piece. Two assymetrical players compete: the ***attacker*** and the ***defender***. A turn of the game goes the following way :

- The attacker separates the pieces in two sets $A$ and $B$ both with at least one piece.
- The defender chooses one set to destroy and all pieces of the remaining set move up of one level.

The attacker wins if **one (ore more) piece reaches the top**, and the defender wins if **all pieces are removed** or it remains only one piece on a level strictly below $K$.

The following picture, taken from the paper, illustrates this:

<img src='schema_ess.PNG'>

## Environment specificities

ESS games offer a nice field to experiment with RL techniques, because there exists several theoretical results which make evaluation of performance easier.

Let us start with a definition.

**Definition : potential function**

Let us consider a state $\mathcal{S} = (n_1, ..., n_k)$ where $n_i$ is the number of pieces in level $i$. We define then the ***potential*** of $\mathcal{S}$, noted $\phi(\mathcal{S})$ by the following function:

$$\phi(\mathcal{S}) = \sum_{i=1}^{K}n_i2^{-(K-i)}$$

**Reminder** : $K$ represents the number of levels.

The potential is a key metric for results and strategies of ESS games.


The possible outcomes and optimal strategies are in the following theorems:

**Theorem 1**

Let an instance of the ESS game with $N$ pieces and $K$ levels randomly placed on the board, and $\mathcal{S}_0$ the initial state.

- If $\phi(\mathcal{S}_0)<1$, the defender can always win
- If $\phi(\mathcal{S}_0)\geq1$, the attacker can always win

If all pieces start at the bottom, this condition $\phi(\mathcal{S}_0)<1$ transforms to $N<2^K$.

**Theorem 2**

The optimal strategy for the defender is the following:

- For the two proposed parts $A$ and $B$, compute $\phi(A)$ and $\phi(B)$.
- Eliminate the part with the highest potential.


For the rest of the work we will call *random* policy the policy where the defender chooses randomly between the sets $A$ and $B$, and the *optimal* policy the policy where it plays in an optimal way.

## Our approach

While the authors decided to create an environment based on the ESS using **[gym](https://gym.openai.com/)** (an **openAI** library for developing and comparing reinforcement learning algorithms on famous environments like **Cartpole** or **Pong**) and compare openAI algorithms on the new environment, we decided not to follow their path.

We didn't think it was interesting to develop a gym-compatible ESS environment, and then use the already-made RL algorithms like DQN, PPO or A2C like what was done by the authors. It would have required some time to get use to the gym library and potentially come across errors unrelated to Reinforcement Learning.

We chose then to develop our own RL algorithm on a non-gym ESS environment, that we would thus create from scratch and by doing so, allow us to focus on our algorithm.

We chose to start by **training the defender**, and we suppose that the attacker always plays randomly.

## Algorithms

We implemented the Monte-Carlo method for policy evaluation and policy control without exploring start. The general algorithm is the following :

<img src='MC_RL_algo.png' width="600" height="400">

We slightly adapted this algorithm for the purpose of the game. For instance, it was difficult to enumerate every pair (state,action) from the begining due to the high number of possibilities. Furthermore, only 2 actions are available at every round for the defender, much less that all the theoretical pairs.

The changes were then the following :

- Not initializing Q for every $(s,a)$, but filled at the end of each game
- For updating $\pi$:
    1. Look if the pairs $(s,a)$ and $(s,b)$ are in the table
        * If both are in the table choose $argmax_{a,b} \{Q(s,a), Q(s,b)\}$
        * If one is missing choose the one who is present
        * If both are missing, choose one randomly
    2. Play the chosen action with probability $1-\epsilon$ and the other with probability $\epsilon$

## Code execution

In [1]:
import numpy as np
import random
import pickle
from time import time
from ESS import Game
import matplotlib.pyplot as plt

   
def hash_action(action):
    '''Returns the action as a string to make a dictionnary key'''
    return str(action)
              
def policy(Q, state, split_a, split_b, epsilon=0.1, play_random=False, play_optimal=False):
    '''Play the policy'''
    
    #If we play random, choose a random part
    if play_random:
        if np.random.uniform()>0.5:
            return split_a
        else:
            return split_b
    #If we play optimal, return the part with highest potential  
    if play_optimal:
        weights = 0.5 ** (np.arange(len(split_a)))
        pot_a = np.sum(weights*split_a)
        pot_b = np.sum(weights*split_b)
        if pot_a>=pot_b:
            return split_a
        else:
            return split_b
        
    #Get the keys of the actions to look in the table
    key_a = hash_action(split_a)
    key_b = hash_action(split_b)
    bigger = split_a
    smaller = split_b
    if np.random.uniform()>0.5:
        bigger = split_b
        smaller = split_a
    
    if state in Q.keys():
        if key_a in Q[state].keys():
            if key_b in Q[state].keys():
                if Q[state][key_a] >= Q[state][key_b]:
                    bigger = split_a
                    smaller = split_b
                else:
                    bigger = split_b
                    smaller = split_a
            else:
                bigger = split_a
                smaller = split_b
        else:
            if key_b in Q[state].keys():
                bigger = split_a
                smaller = split_b
                
    #Whatever choice, flip it with epsilon probability          
    if np.random.uniform()>epsilon:
        return bigger
    else:
        return smaller
    
def evaluate_policy(N=1000, random=False, optimal=False, init_bottom=False):
    winner_defender = np.zeros(N)
    for i in range(N):
        game.reset(init_bottom=init_bottom)
        winner=None
        while winner != 'attacker' and winner != 'defender':
            a,b = game.play_attacker()
            action = policy(Q, game.getHash(), a, b, play_random=random, play_optimal=optimal)
            assert (action==a).all() or (action==b).all()
            game.play_defender(action)
            winner = game.winner()
            if winner=='defender':
                winner_defender[i] = 1
    return winner_defender

The following part plays the game, using the game environment we made in the [ESS.py](https://github.com/Adanel/ReinforcementLearningProject/blob/master/ESS.py) file. We initiate the game with a special condition which we highlighted before in the **Theorem 1** so that our defender **can always win**. We are in the case where all pieces start at the bottom. By doing so, we remove the hazard where our defender could loose whereas he plays optimally simply because of the pieces' initialisation. Thus the optimal policy represents 100% games won.

In [11]:
#Initialize the game
K=8 #Levels
M=50 #Pieces
game = Game(K, M, init_bottom=True)

Q = {}
returns = {}

gamma = 0.9

#Number of Monte-Carlo iterations
N = 10000

defender_wins = []#Evaluation of the policy during the training

for i in range(N):
    
    list_of_states = []
    list_of_actions = []
    list_of_rewards = []
    list_of_returns = []
    winner = None
    game.reset(init_bottom=True)
    
    #play the game
    while winner != 'attacker' and winner != 'defender':
        a,b = game.play_attacker()
        action = policy(Q, game.getHash(), a, b, epsilon=0.3)
        assert (action==a).all() or (action==b).all()
        list_of_states.append(game.getHash())
        list_of_actions.append(hash_action(action))
        game.play_defender(action)
        winner = game.winner()
        if winner =='attacker':
            list_of_rewards.append(-1)
        elif winner=='defender':
            list_of_rewards.append(1)
        else:
            list_of_rewards.append(0)
    
    assert len(list_of_states) == len(list_of_rewards)
    
    #compute the returns
    n = len(list_of_states)      
    list_of_returns.append(0)
    for j in range(1,n+1):
        list_of_returns.append(list_of_rewards[n-j] + gamma * list_of_returns[j-1])
    list_of_returns.reverse()
    
    assert len(list_of_states) == len(list_of_returns)-1
    assert list_of_returns[-1]==0
    #add returns to dictionnary
    for j,s in enumerate(list_of_states):
        a = list_of_actions[j]
        if s in returns.keys():
            if a in returns[s].keys():
                returns[s][a].append(list_of_returns[j])
            else:
                returns[s][a] = [list_of_returns[j]]
        else:
            returns[s] = {}
            returns[s][a] = [list_of_returns[j]]
            
    #Every 500 steps, update the policy       
    if i%500==0:
        for s in returns.keys():
            Q[s] = dict()
            for a in returns[s].keys():
                Q[s][a] = np.mean(returns[s][a])
        #keep results for plots, keep commented if not interested by the plots, it takes lots of times
        #defender_wins.append(np.sum(evaluate_policy(init_bottom=True)))
        
        
random_rate = np.sum(evaluate_policy(random=True, init_bottom=True))/10
with_training_rate = np.sum(evaluate_policy(init_bottom=True))/10
optimal_rate = np.sum(evaluate_policy(optimal=True, init_bottom=True))/10
        

#Check the result of training by comparing to random and optimal policy
print("In random case, defender wins ", random_rate, "% times")  
print("After training, defender wins ", with_training_rate, "% times")
print("In optimal case, defender wins ", optimal_rate, "% times")

In random case, defender wins  89.9 % times
After training, defender wins  99.3 % times
In optimal case, defender wins  100.0 % times


The following part can be used to display the progress of the winrate following the training. Because our algorithm takes time to learn depending on the Game's conditions, we also took screenshots of the plots and we show them underneath. 

In [ ]:
#Code to plot results
  
#plt.figure()
#plt.plot(np.arange(0,N,500), np.array(defender_wins)/10, label='learned policy')
#plt.hlines(random_rate, 0, N, colors = 'g', label = "random policy")
#plt.hlines(with_training_rate, 0, N, colors = 'b', label = 'learned policy')
#plt.hlines(optimal_rate, 0, N, colors = 'r', label = 'optimal policy')
#plt.xlabel("Steps of training", fontsize='large')
#plt.ylabel("% of victory in 1000 games", fontsize='large')
#plt.legend()
#plt.axis([0,N,0,110])
#plt.title("Performance for "+str(K)+ " levels and "+str(M)+" pieces", fontsize='x-large')

## Results

Here are displayed the performance of our algorithm in different configurations. One can see that for few pieces, the random strategy is already very good, and the algorithm has very few to learn. For more pieces, the convergence is much slower, and we don't know if it converges at all.

<img src='training_plot_5_8_10000_bottom.svg?sanitize=true' width="600" height="400">

<img src='training_plot_5_20_10000_bottom.svg?sanitize=true' width="600" height="400">

<img src='training_plot_8_500_60000_bottom.svg?sanitize=true' width="600" height="400">